# Express

Given A and C's (random) positions around the campfire, the laser fired between them divides the remaining campers into two groups:

In order for the beams *not* to cross, we need both B and D to be in the same group. The probability of this (conditional on the clockwise distance between A and C) is pretty straightforward to calculate.

In [8]:
def cond_prb(dist):
    size_left_group = dist - 1
    size_right_group = 20 - dist - 1
    prb_left_group = (size_left_group / 18) * ((size_left_group - 1) / 17) # both in left
    prb_right_group = (size_right_group / 18) * ((size_right_group - 1) / 17) # both in right
    return(prb_left_group + prb_right_group) # either

Now we have the conditional probability, so we just need to average over all of the 19 positions that C could be in relative to A:

In [6]:
sum(map(cond_prb, range(1,20))) / 19

0.6666666666666665

So there is a 2/3 chance of *not* ending the world, and a 1/3 chance of ending it. Could be worse!

# Classic

This one is tougher and is probably going to test my geometry and probability skills.

## Part 1: two breaks, any triangle

Say we have a stick of length one, with positions on the stick indexed by $(0,1)$. For any two random (uniformly-distributed) breaks on the stick, call the one on the left $x$ and the one on the right $y$. Then, if we imagine each break acting like a hinge, we know we can form a triangle if the arcs traced out by the left and right branches intersect, like so:

One thing this diagram makes clear right away is that we need $x$ and $y$ to stay on their respective 'sides' of the line (so we can't have $x > 1/2$ or $y < 1/2$). Otherwise one of the arcs will be too big.

I'm going to start by talking about what we need for the position of $y$ relative to $x$, and then talk about the distribution of $x$ itself.

**Probability of making a triangle conditional on $x$**

We can state the 'intersection' condition w.r.t. the lengths of the branches. First, note that the location of the leftmost point in $y$'s arc is given by $1-2(1-y) = 2y-1$. Then what we need is

$$0 < 2y-1 < 2x$$

or, equivalently:

$$y \in \Big( \frac{1}{2}, \frac{1}{2} + x \Big).$$

So, given the position of $x$, $y$ needs to be in an area of size $x$. Since $y$ is always to the right of $x$, the total size of the area that it can be in is $1-x$. Since it is (conditionally) uniformly-distributed within this area, the probability of creating a triangle is then given by

$$ Pr(\Delta | x) = \frac{x}{1-x} \quad \textrm{for} \quad x \in \Big( 0, \frac{1}{2} \Big) .$$

**The distribution of $x$**

We're looking for the density function $f$ of a variable $x = \min(z_1, z_2)$ where $z_1, z_2$ are uniformly-distributed. We can find the CDF first 

$$F(x) = Pr(p \leq x) = 1 - Pr(p > x) = 1 - (1-x)^2 $$

and then get the PDF by taking the first derivative: $f(x) = 2(1-x)$.

**Unconditional probability of making a triangle**

$$\int{Pr(\Delta | x) \cdot f(x)} \; dx = \int_0^{1/2}{\frac{x}{1-x} \cdot 2(1-x)} \; dx = \frac{1}{4}$$.


## Part 2: three sticks, any triangle

We have a very similar diagram as before, but now with $a$, $b$, and $c$ denoting sticks each uniformly distributed on $(0,1)$.

My intuition here is that we'll be able to form a triangle iff the longest side is no longer than the sum of the two shorter sides. This seems a bit tricky to do analytically so I'm just going to simulate it. I'll create an $n \times 3$ matrix, sort each row and then test whether each element is smaller than the sum of the other two.

In [83]:
import numpy as np
np.random.seed(1)
n = 1000000

sticks = np.random.rand(n,3)

for i in range(n):
    sticks[i] = np.sort(sticks[i])

triangle = list()
for row in sticks:
    triangle.append(row[0] + row[1] > row[2])

print("triangle:", sum(triangle) / len(triangle))

triangle: 0.499937


I'll take a stab in the dark and guess that the true probability is $1/2$. While I'm simulating, I might as well do Part 4 (for an acute triangle) as well. If $c$ is the longest side, the condition for an *acute* triangle is that $a^2+b^2>c^2$. So just need a slight modification of the earlier test.

## Part 4, three sticks, acute triangle

In [84]:
acute = list()
for row in sticks:
    row = row**2
    acute.append(row[0] + row[1] > row[2])

print("acute:", sum(acute) / len(acute))

acute: 0.214737


Not so even of a value...21.47%.

## Part 3: two breaks, acute triangle

Going to stick with simulation. Similar to before, I'll use random $x$ and $y$ to calculate side lengths $a$, $b$, and $c$, where $c$ is the longest.

In [85]:
np.random.seed(123)
n = 1000000

breaks = np.random.rand(n,2)

for i in range(n):
    breaks[i] = np.sort(breaks[i])

sticks = np.column_stack((breaks[:,0],
                          breaks[:,1] - breaks[:,0],
                          1 - breaks[:,1]))

for i in range(n):
    sticks[i] = np.sort(sticks[i])

triangle = list()
for row in sticks:
    triangle.append(row[0] + row[1] > row[2])

print("triangle:", sum(triangle) / len(triangle))

acute = list()
for row in sticks:
    row = row**2
    acute.append(row[0] + row[1] > row[2])

print("acute:", sum(acute) / len(acute))

triangle: 0.2500134
acute: 0.0794213


I also checked my math on Part 1: looks like $1/4$ is correct for any triangle. For an acute triangle, it is 7.94% and I suspect this might be $1/4\pi$.